In [1]:
import numpy as np
from numba import njit, prange
from math import cos, sin

@njit(parallel=True)
def __phi_prime_OECS(X, Y, phi, s, S11_x, S11_y, S12_x, S12_y, S22_x, S22_y, Ncores):
    
    phi_prime = np.zeros((Y.shape[0], X.shape[1], phi.shape[0]))
    
    for i in prange(Y.shape[0]):
        
        for j in prange(X.shape[1]):
            
            s_ = s[i, j]
            
            if np.abs(s_) > 0 and np.isfinite(s_) and np.isfinite(S11_x[i, j]) and np.isfinite(S11_y[i, j]) and np.isfinite(S12_x[i, j]) and np.isfinite(S12_y[i, j]) and np.isfinite(S22_x[i, j]) and np.isfinite(S22_y[i, j]):
                
                s_matrix = s_*np.eye(2, 2)
            
                S11_x_ = S11_x[i, j]
                S11_y_ = S11_y[i, j]
            
                S12_x_ = S12_x[i, j]
                S12_y_ = S12_y[i, j]
            
                S22_x_ = S22_x[i, j]
                S22_y_ = S22_y[i, j]
            
                for p in prange(phi.shape[0]):
                
                    phi_ = phi[p]
                
                    e = np.array([cos(phi_), sin(phi_)])
                    v = np.array([-sin(phi_), cos(phi_)])
                
                    grad_S_e = np.array([[S11_x_*cos(phi_)+S11_y_*sin(phi_), S12_x_*cos(phi_)+S12_y_*sin(phi_)], [S12_x_*cos(phi_)+S12_y_*sin(phi_), S22_x_*cos(phi_)+S22_y_*sin(phi_)]])
                
                    #K = sin(phi_)*cos(phi_)**2*(-S11_x_+S12_y_+S22_x_)+sin(phi_)**2*cos(phi_)*(-S11_y_-S12_x_+S22_y_)+S12_x_*cos(phi_)**3-S12_y_*sin(phi_)**3
    
                    K = v.T@(grad_S_e@e)
        
                    phi_prime[i, j, p] = 1/(2*s_)*K#*(v.T@(grad_C_e@e))
                    
            else:
                
                phi_prime[i, j, :] = 0
      
    return phi_prime

In [2]:
def _phi_prime_OECS(X, Y, s, S11_x, S11_y, S12_x, S12_y, S22_x, S22_y, Ncores):
    
    phi = np.linspace(0, 2*np.pi, 180)
    
    from scipy.interpolate import RegularGridInterpolator
    
    phi_prime = __phi_prime_OECS(X, Y, phi, s, S11_x, S11_y, S12_x, S12_y, S22_x, S22_y, Ncores)
    
    return RegularGridInterpolator((Y[:,0], X[0,:], phi), phi_prime)